In [420]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf

# Function to calculate CAGR
def calculate_cagr(start_value, end_value, periods):
    if start_value == 0 or periods <= 0:
        return 0
    return ((end_value / start_value) ** (1 / periods) - 1) * 100

# Main function to fetch and compute metrics
def get_peaceful_investing_data_fmp(ticker, api_key, years=10):
    base_url = "https://financialmodelingprep.com/api/v3"
    
    # Fetch data with error handling
    income_response = requests.get(f"{base_url}/income-statement/{ticker}?limit={years}&apikey={api_key}")
    if income_response.status_code != 200:
        return f"Error fetching income data: {income_response.text}"
    income = income_response.json()
    
    balance_response = requests.get(f"{base_url}/balance-sheet-statement/{ticker}?limit={years}&apikey={api_key}")
    if balance_response.status_code != 200:
        return f"Error fetching balance data: {balance_response.text}"
    balance = balance_response.json()
    
    cashflow_response = requests.get(f"{base_url}/cash-flow-statement/{ticker}?limit={years}&apikey={api_key}")
    if cashflow_response.status_code != 200:
        return f"Error fetching cashflow data: {cashflow_response.text}"
    cashflow = cashflow_response.json()
    
    key_metrics_response = requests.get(f"{base_url}/key-metrics/{ticker}?limit={years}&apikey={api_key}")
    if key_metrics_response.status_code != 200:
        return f"Error fetching key metrics: {key_metrics_response.text}"
    key_metrics = key_metrics_response.json()
    
    profile_response = requests.get(f"{base_url}/profile/{ticker}?apikey={api_key}")
    if profile_response.status_code != 200:
        return f"Error fetching profile: {profile_response.text}"
    profile_data = profile_response.json()
    if not profile_data:
        return "No profile data available for this ticker"
    profile = profile_data[0]
    
    quote_response = requests.get(f"{base_url}/quote/{ticker}?apikey={api_key}")
    if quote_response.status_code != 200:
        return f"Error fetching quote: {quote_response.text}"
    quote_data = quote_response.json()
    if not quote_data:
        return "No quote data available for this ticker"
    quote = quote_data[0]
    
    if not income or not balance or not cashflow:
        return "Incomplete financial data fetched"
    
    return income, balance, cashflow, key_metrics, profile, quote

def compute_metrics_fmp(income, balance, cashflow, key_metrics, profile, quote):

    # Convert to DataFrames (reverse to chronological: oldest first)
    income_df = pd.DataFrame(income[::-1])
    balance_df = pd.DataFrame(balance[::-1])
    cashflow_df = pd.DataFrame(cashflow[::-1])
    key_metrics_df = pd.DataFrame(key_metrics[::-1])
    
    # Current year index (latest)
    latest = -1
    
    # Compute metrics with defaults for missing data
    metrics = {}
    metrics['ticker'] = income_df['symbol'][0]
    # Return on Equity (ROE) - Latest and 3-year average
    net_income = income_df['netIncome']
    equity = balance_df['totalStockholdersEquity'].shift(1)
    metrics['ROE'] = (net_income.iloc[latest] / equity.iloc[latest] * 100) if equity.iloc[latest] != 0 else 0
    roe_3yr = [(net_income.iloc[i] / equity.iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if equity.iloc[i] != 0]
    metrics['ROE (3yr avg)'] = np.mean(roe_3yr) if roe_3yr else 0
    
    # Return on Capital Employed (ROCE) - Latest and 3-year average
    ebit = income_df['operatingIncome']
    capital_employed = balance_df['totalAssets'] - balance_df['totalCurrentLiabilities']
    metrics['ROCE'] = (ebit.iloc[latest] / capital_employed.iloc[latest] * 100) if capital_employed.iloc[latest] != 0 else 0
    roce_3yr = [(ebit.iloc[i] / capital_employed.iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if capital_employed.iloc[i] != 0]
    metrics['ROCE (3yr avg)'] = np.mean(roce_3yr) if roce_3yr else 0
    
    # Net Fixed Assets (NFA) for each year
    net_fixed_assets = balance_df['propertyPlantEquipmentNet']
    
    # Net Fixed Asset Turnover (NFAT) - Calculate for each year, then 3-year average
    revenue = income_df['revenue']
    nfat_values = []
    for i in range(1, len(income_df)):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
        nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    if len(income_df) == 1:
        avg_nfa_year = net_fixed_assets.iloc[latest]
        nfat_year = revenue.iloc[latest] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    metrics['NFAT'] = nfat_values[-1] if nfat_values else 0  # Latest year's NFAT
    nfat_3yr = nfat_values[-min(3, len(nfat_values)):]
    metrics['NFAT (3yr avg)'] = np.mean(nfat_3yr) if nfat_3yr else metrics['NFAT']  # Fallback to latest if <3
    
    # Net Profit Margin (NPM) - 3-year average
    npm_3yr = [(income_df['netIncome'].iloc[i] / income_df['revenue'].iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if income_df['revenue'].iloc[i] != 0]
    metrics['NPM'] = np.mean(npm_3yr) if npm_3yr else 0
    
    # Dividend Payout Ratio (DPR) - 3-year average
    dividends_paid = cashflow_df['dividendsPaid'].abs()
    dpr_3yr = [(dividends_paid.iloc[i] / income_df['netIncome'].iloc[i] * 100) for i in range(-min(3, len(income_df)), 0) if income_df['netIncome'].iloc[i] != 0]
    metrics['DPR'] = np.mean(dpr_3yr) if dpr_3yr else 0
    
    # Retention Ratio (1 - DPR)
    metrics['Retention Ratio'] = (1 - (metrics['DPR'] / 100)) * 100
    
    # Depreciation Rate (Dep as % of NFA) - 3-year average
    dep = cashflow_df['depreciationAndAmortization']
    dep_3yr = []
    for i in range(-min(3, len(income_df)), 0):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.shift(1).iloc[i]) / 2 if len(balance_df) > 1 and i > -len(income_df) + 1 else net_fixed_assets.iloc[i]
        dep_year = (dep.iloc[i] / avg_nfa_year * 100) if avg_nfa_year != 0 else 0
        dep_3yr.append(dep_year)
    metrics['Dep'] = np.mean(dep_3yr) if dep_3yr else 0
    
    # Self Sustainable Growth Rate (SSGR) = NFAT * NPM * (1 - DPR) - Dep
    npm_decimal = metrics['NPM'] / 100
    dpr_decimal = metrics['DPR'] / 100
    dep_decimal = metrics['Dep'] / 100
    metrics['SSGR'] = (metrics['NFAT (3yr avg)'] * npm_decimal * (1 - dpr_decimal) - dep_decimal) * 100  # As %
    
    # Avg NPM (over 3 years) - Already handled above
    metrics['Av NPM (over 3 years)'] = metrics['NPM']
    
    # Avg NFAT (over 3 years) - Already handled above
    metrics['Au NFA/T (over 3 years)'] = metrics['NFAT (3yr avg)']
    
    # Avg Dep % NFA (over 3 years) - Already handled above
    metrics['Av Dep%NFA (over 3 years)'] = metrics['Dep']
    
    # Avg Retention Ratio (over 3 years)
    ret_3yr = [(1 - (dividends_paid.iloc[i] / income_df['netIncome'].iloc[i])) * 100 for i in range(-min(3, len(income_df)), 0) if income_df['netIncome'].iloc[i] != 0]
    metrics['Au Retention ratio (over 3 years)'] = np.mean(ret_3yr) if ret_3yr else 0
    
    # Debt to Equity (d/e)
    metrics['d/e'] = key_metrics_df['debtToEquity'].iloc[latest] if 'debtToEquity' in key_metrics_df.columns and not key_metrics_df.empty else (balance_df['totalDebt'].iloc[latest] / balance_df['totalStockholdersEquity'].iloc[latest] if balance_df['totalStockholdersEquity'].iloc[latest] != 0 else 0)
    
    # Interest Coverage
    interest_exp = abs(income_df['interestExpense'].iloc[latest])  # Make positive
    metrics['Interest coverage'] = (income_df['operatingIncome'].iloc[latest] / interest_exp) if interest_exp != 0 else float('inf')
    
    # Tax %
    metrics['tax %'] = (income_df['incomeTaxExpense'].iloc[latest] / income_df['incomeBeforeTax'].iloc[latest] * 100) if 'incomeBeforeTax' in income_df.columns and income_df['incomeBeforeTax'].iloc[latest] != 0 else 0
    
    # Cumulative PAT (cPAT) - Sum over last 5 years
    metrics['cPAT'] = income_df['netIncome'][-5:].sum() if len(income_df) >= 5 else income_df['netIncome'].sum()
    
    # CFO (latest)
    metrics['CFO'] = cashflow_df['netCashProvidedByOperatingActivities'].iloc[latest]
    
    # Cumulative CFO (cCFO)
    metrics['cCFO'] = cashflow_df['netCashProvidedByOperatingActivities'][-5:].sum() if len(cashflow_df) >= 5 else cashflow_df['netCashProvidedByOperatingActivities'].sum()
    
    # Cumulative CFO / cPAT
    metrics['cCFO/cPAT'] = (metrics['cCFO'] / metrics['cPAT']) if metrics['cPAT'] != 0 else 0
    
    # ROA (p/a)
    metrics['p/a'] = (income_df['netIncome'].iloc[latest] / balance_df['totalAssets'].shift(1).iloc[latest] * 100) if balance_df['totalAssets'].shift(1).iloc[latest] != 0 else 0
    
    # Price to Earnings (p/e)
    pe = quote.get('pe', float('inf'))
    metrics['p/e'] = pe
    
    # Earnings Yield (EY)
    metrics['EY'] = (income_df['eps'].iloc[latest] / quote['price'] * 100) if quote['price'] != 0 else 0
    
    # Earnings Growth 5yr CAGR
    eps_values = income_df['eps'][-6:] if len(income_df) >= 6 else income_df['eps']
    periods = len(eps_values) - 1
    metrics['Earnings Growth 5yr cagr'] = calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods) if periods > 0 else 0
    
    # PEG
    metrics['PEG'] = (pe / metrics['Earnings Growth 5yr cagr']) if metrics['Earnings Growth 5yr cagr'] != 0 else float('inf')
    
    # No. shares (cr) - in crores
    shares_out = profile.get('sharesOutstanding', quote['marketCap'] / quote['price'] if quote['price'] != 0 else 0)
    metrics['no. shares (cr)'] = shares_out / 1e7
    
    # Price to Sales (p/s)
    metrics['p/s'] = quote.get('priceToSalesRatio', 0)
    
    # NFA + CWIP
    cwip = balance_df.get('constructionInProgress', pd.Series([0]*len(balance_df))).iloc[latest]
    metrics['NFA + CWIP'] = net_fixed_assets.iloc[latest] + cwip
    
    # Capex = (NFA + CWIP end) - (NFA + CWIP start) + Dep
    if len(balance_df) >= 2:
        nfa_cwip_end = net_fixed_assets.iloc[latest] + cwip
        nfa_cwip_start = net_fixed_assets.iloc[latest-1] + balance_df.get('constructionInProgress', pd.Series([0]*len(balance_df))).iloc[latest-1]
        metrics['Capex'] = nfa_cwip_end - nfa_cwip_start + dep.iloc[latest]
    else:
        metrics['Capex'] = abs(cashflow_df['capitalExpenditure'].iloc[latest])  # Fallback
    
    # Free Cash Flow (FCF) = CFO - Capex
    metrics['FCF'] = metrics['CFO'] - metrics['Capex']
    
    # FCF%
    metrics['FCF%'] = (metrics['FCF'] / net_income.iloc[latest] * 100) if net_income.iloc[latest] != 0 else 0
    
    # Dividend Yield (DV)
    per_share_div = (dividends_paid.iloc[latest] / shares_out) if shares_out != 0 else 0
    metrics['DV'] = (per_share_div / quote['price'] * 100) if quote['price'] != 0 else 0
    
    # Mcap (cr)
    metrics['Mcap (cr)'] = quote['marketCap'] / 1e7
    
    # d/e decreasing trend 5 yrs
    de_ratios = [balance_df['totalDebt'].iloc[i] / balance_df['totalStockholdersEquity'].iloc[i] if balance_df['totalStockholdersEquity'].iloc[i] != 0 else 0 for i in range(-5, 0) if len(balance_df) >= 5]
    metrics['d/e decreasing trend 5 yrs'] = all(de_ratios[i] > de_ratios[i+1] for i in range(len(de_ratios)-1)) if len(de_ratios) > 1 else False
    
    # Financial Analysis Criteria
    sales_values = income_df['revenue'][-6:] if len(income_df) >= 6 else income_df['revenue']
    sales_periods = len(sales_values) - 1
    sales_cagr = calculate_cagr(sales_values.iloc[0], sales_values.iloc[-1], sales_periods) if sales_periods > 0 else 0
    metrics['Sales cagr >15%'] = sales_cagr > 15
    metrics['npm >8%'] = metrics['NPM'] > 8
    metrics['Tax payout >25%'] = metrics['tax %'] > 25
    metrics['Interest coverage >3'] = metrics['Interest coverage'] > 3
    metrics['d/e <0.5'] = metrics['d/e'] < 0.5
    metrics['CFO >0'] = metrics['CFO'] > 0
    metrics['net cash flow positive'] = cashflow_df['netChangeInCash'].iloc[latest] > 0
    metrics['cCFO > PAT'] = metrics['cCFO/cPAT'] > 1
    
    # Valuation Analysis
    metrics['p/e <10'] = pe < 10
    metrics['peg <1'] = metrics['PEG'] < 1
    metrics['EY >7%'] = metrics['EY'] > 7
    metrics['p/b <3'] = quote.get('priceToBookRatio', float('inf')) < 3
    metrics['DV >3%'] = metrics['DV'] > 3
    
    # Margin of Safety
    metrics['EY >7'] = metrics['EY'] > 7  # Duplicate
    metrics['sgr > Sales growth (very linear)'] = metrics['SSGR'] > sales_cagr
    metrics['FCF/CFO'] = (metrics['FCF'] / metrics['CFO']) if metrics['CFO'] != 0 else 0
    
    # Current Price
    metrics['Current Price'] = quote['price']

    # Raw Financial Data
    # Raw Financial Data (corrected columns)
    metrics['Market Cap'] = quote.get('marketCap', 0)  # In original currency
    metrics['Net Income'] = net_income.iloc[latest] if not pd.isna(net_income.iloc[latest]) else 0
    metrics['Total Revenue'] = revenue.iloc[latest] if not pd.isna(revenue.iloc[latest]) else 0
    metrics['Total Assets'] = balance_df['totalAssets'].iloc[latest] if not pd.isna(balance_df['totalAssets'].iloc[latest]) else 0
    metrics['Total Liabilities'] = balance_df['totalLiabilities'].iloc[latest] if not pd.isna(balance_df['totalLiabilities'].iloc[latest]) else 0
    metrics['Total Stockholders Equity'] = equity.iloc[latest] if not pd.isna(equity.iloc[latest]) else 0
    metrics['Total Debt'] = balance_df['totalDebt'].iloc[latest] if not pd.isna(balance_df['totalDebt'].iloc[latest]) else 0
    metrics['Cash and Cash Equivalents'] = balance_df['cashAndCashEquivalents'].iloc[latest] if not pd.isna(balance_df['cashAndCashEquivalents'].iloc[latest]) else 0
    metrics['Current Assets'] = balance_df['totalCurrentAssets'].iloc[latest] if not pd.isna(balance_df['totalCurrentAssets'].iloc[latest]) else 0
    metrics['Current Liabilities'] = balance_df['totalCurrentLiabilities'].iloc[latest] if not pd.isna(balance_df['totalCurrentLiabilities'].iloc[latest]) else 0
    metrics['Operating Cash Flow'] = metrics['CFO']  # Already calculated, stored raw
    metrics['Capital Expenditure'] = abs(cashflow_df['capitalExpenditure'].iloc[latest]) if not pd.isna(cashflow_df['capitalExpenditure'].iloc[latest]) else 0
    metrics['Dividends Paid'] = dividends_paid.iloc[latest] if not pd.isna(dividends_paid.iloc[latest]) else 0
    metrics['Depreciation & Amortization'] = dep.iloc[latest] if not pd.isna(dep.iloc[latest]) else 0
    metrics['Interest Expense'] = interest_exp if not pd.isna(interest_exp) else 0
    metrics['Income Tax Expense'] = income_df['incomeTaxExpense'].iloc[latest] if not pd.isna(income_df['incomeTaxExpense'].iloc[latest]) else 0
    metrics['Shares Outstanding'] = shares_out
    metrics['Current Price'] = quote['price']  # Already calculated, stored raw
    metrics['Net Fixed Assets'] = net_fixed_assets.iloc[latest] if not pd.isna(net_fixed_assets.iloc[latest]) else 0
    metrics['Construction in Progress'] = cwip if not pd.isna(cwip) else 0
    metrics['Net Debt'] = metrics['Total Debt'] - metrics['Cash and Cash Equivalents'] if not pd.isna(metrics['Total Debt']) and not pd.isna(metrics['Cash and Cash Equivalents']) else 0



    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])

    #metrics_df.to_csv('fmp/aapl.csv')
    metrics_df.to_csv(f'fmp/{metrics['ticker']}.csv')
    
    return metrics


def retrieve_data_from_yfinance(ticker):
    stock = yf.Ticker(ticker)

    return stock

# Main function to fetch and compute metrics
def get_peaceful_investing_metrics_from_yfinance(stock, years=5):  # Default to 5 years as yfinance often provides ~4-5
    #stock = yf.Ticker(ticker)
    # Fetch data
    income = stock.financials.transpose()  # Transpose to rows=years, columns=items; recent first
    balance = stock.balance_sheet.transpose()
    cashflow = stock.cashflow.transpose()
    info = stock.info  # Dict for quote/profile/metrics
    
    # Earnings history for EPS/Revenue (annual)
    #earnings = stock.earnings  # pd.DataFrame with Revenue, Earnings
    
    if income.empty or balance.empty or cashflow.empty:
        return "Error fetching data for this ticker"
    
    # Reset index to make date a column; sort oldest first for calculations

    

    income = income.reset_index().rename(columns={'index': 'Date'})
    balance = balance.reset_index().rename(columns={'index': 'Date'})
    cashflow = cashflow.reset_index().rename(columns={'index': 'Date'})
    
    income['Date'] = pd.to_datetime(income['Date'])
    balance['Date'] = pd.to_datetime(balance['Date'])
    cashflow['Date'] = pd.to_datetime(cashflow['Date'])

    income = income.sort_values('Date')
    balance = balance.sort_values('Date')
    cashflow = cashflow.sort_values('Date')
    # Current year index (latest)
    latest = -1
    
    # Available years
    num_years = len(income)
    
    # Compute metrics with defaults for missing data
    metrics = {}
    metrics['ticker'] = stock.ticker
    # Return on Equity (ROE) - Latest and 3-year average
    net_income = income.get('Net Income', pd.Series([0]*num_years))
    equity = balance.get('Stockholders Equity', pd.Series([0]*num_years)).shift(1)
    #metrics['ROE'] = (net_income.iloc[latest] / equity.iloc[latest] * 100) if equity.iloc[latest] != 0 else 0
    #roe_3yr = [(net_income.iloc[i] / equity.iloc[i] * 100) for i in range(-min(3, num_years), 0) if equity.iloc[i] != 0]
    #metrics['ROE (3yr avg)'] = np.mean(roe_3yr) if roe_3yr else 0
    
    # Return on Capital Employed (ROCE) - Latest and 3-year average
    ebit = income.get('EBIT', pd.Series([0]*num_years))
    capital_employed = balance.get('Total Assets', pd.Series([0]*num_years)) - balance.get('Current Liabilities', pd.Series([0]*num_years))
    metrics['ROCE'] = (ebit.iloc[latest] / capital_employed.iloc[latest] * 100) if capital_employed.iloc[latest] != 0 else 0
    roce_3yr = [(ebit.iloc[i] / capital_employed.iloc[i] * 100) for i in range(-min(3, num_years), 0) if capital_employed.iloc[i] != 0]
    metrics['ROCE (3yr avg)'] = np.mean(roce_3yr) if roce_3yr else 0
    
    # Net Fixed Assets (NFA) for each year
    net_fixed_assets = balance.get('Net PPE', pd.Series([0]*num_years))
    
    # Net Fixed Asset Turnover (NFAT) - Calculate for each year, then 3-year average
    revenue = income.get('Total Revenue', pd.Series([0]*num_years))
    nfat_values = []
    for i in range(1, num_years):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
        nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    if num_years == 1:
        avg_nfa_year = net_fixed_assets.iloc[latest]
        nfat_year = revenue.iloc[latest] / avg_nfa_year if avg_nfa_year != 0 else 0
        nfat_values.append(nfat_year)
    metrics['NFAT'] = nfat_values[-1] if nfat_values else 0  # Latest year's NFAT
    nfat_3yr = nfat_values[-min(3, len(nfat_values)):]

    print(f"nfat_3yr : {nfat_3yr}")
    metrics['NFAT (3yr avg)'] = np.mean(nfat_3yr) if nfat_3yr else metrics['NFAT']  # Fallback to latest if <3
    
    # Net Profit Margin (NPM) - 3-year average
    npm_3yr = [(net_income.iloc[i] / revenue.iloc[i] * 100) for i in range(-min(3, num_years), 0) if revenue.iloc[i] != 0]
    metrics['NPM'] = np.mean(npm_3yr) if npm_3yr else 0
    
    # Dividend Payout Ratio (DPR) - 3-year average
    dividends_paid = cashflow.get('Cash Dividends Paid', pd.Series([0]*num_years)).abs()
    dpr_3yr = [(dividends_paid.iloc[i] / net_income.iloc[i] * 100) for i in range(-min(3, num_years), 0) if net_income.iloc[i] != 0]
    metrics['DPR'] = np.mean(dpr_3yr) if dpr_3yr else 0
    
    # Retention Ratio (1 - DPR)
    metrics['Retention Ratio'] = (1 - (metrics['DPR'] / 100)) * 100
    
    # Depreciation Rate (Dep as % of NFA) - 3-year average
    dep = cashflow.get('Depreciation And Amortization', pd.Series([0]*num_years))
    dep_3yr = []
    for i in range(-min(3, num_years), 0):
        avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.shift(1).iloc[i]) / 2 if num_years > 1 and i > -num_years + 1 else net_fixed_assets.iloc[i]
        dep_year = min((abs(dep.iloc[i]) / avg_nfa_year * 100) if avg_nfa_year != 0 else 0, 100)  # Use abs for dep, cap at 100%
        #print(avg_nfa_year)
        if dep_year > 100:
            print(f"Warning: Dep % for year {i} exceeded 100%; capped to 100%. Check data for {ticker}.")
        dep_3yr.append(dep_year)
    metrics['Dep'] = np.mean(dep_3yr) if dep_3yr else 0
    
    # Self Sustainable Growth Rate (SSGR) = NFAT * NPM * (1 - DPR) - Dep
    npm_decimal = metrics['NPM'] / 100
    dpr_decimal = metrics['DPR'] / 100
    dep_decimal = metrics['Dep'] / 100
    metrics['SSGR'] = (metrics['NFAT (3yr avg)'] * npm_decimal * (1 - dpr_decimal) - dep_decimal) * 100  # As %
    
    # Avg NPM (over 3 years) - Already handled above
    metrics['Av NPM (over 3 years)'] = metrics['NPM']
    
    # Avg NFAT (over 3 years) - Already handled above
    metrics['Au NFA/T (over 3 years)'] = metrics['NFAT (3yr avg)']
    
    # Avg Dep % NFA (over 3 years) - Already handled above
    metrics['Av Dep%NFA (over 3 years)'] = metrics['Dep']
    
    # Avg Retention Ratio (over 3 years)
    ret_3yr = [(1 - (dividends_paid.iloc[i] / net_income.iloc[i])) * 100 for i in range(-min(3, num_years), 0) if net_income.iloc[i] != 0]
    metrics['Au Retention ratio (over 3 years)'] = np.mean(ret_3yr) if ret_3yr else 0

    # Debt to Equity (d/e)
    total_debt = balance.get('Total Debt', pd.Series([0]*num_years)).iloc[latest]
    metrics['d/e'] = (total_debt / equity.iloc[latest]) if equity.iloc[latest] != 0 else 0
    
    # Interest Coverage
    interest_exp = abs(income.get('Interest Expense', pd.Series([0]*num_years)).iloc[latest])
    metrics['Interest coverage'] = (income.get('Operating Income', pd.Series([0]*num_years)).iloc[latest] / interest_exp) if interest_exp != 0 else float('inf')
    
    # Tax %
    tax_exp = income.get('Income Tax Expense', pd.Series([0]*num_years)).iloc[latest] if 'Income Tax Expense' in income.columns else income.get('Tax Provision', pd.Series([0]*num_years)).iloc[latest] if 'Tax Provision' in income.columns else 0
    pretax = income.get('Pretax Income', pd.Series([0]*num_years)).iloc[latest]
    metrics['tax %'] = (tax_exp / pretax * 100) if pretax != 0 else 0
    
    # Cumulative PAT (cPAT) - Sum over available years (up to 5)
    metrics['cPAT'] = net_income[-min(5, num_years):].sum()
    
    # CFO (latest)
    metrics['CFO'] = cashflow.get('Operating Cash Flow', pd.Series([0]*num_years)).iloc[latest]
    
    # Cumulative CFO (cCFO)
    metrics['cCFO'] = cashflow.get('Operating Cash Flow', pd.Series([0]*num_years))[-min(5, num_years):].sum()
    
    # Cumulative CFO / cPAT
    metrics['cCFO/cPAT'] = (metrics['cCFO'] / metrics['cPAT']) if metrics['cPAT'] != 0 else 0
    
    # ROA (p/a)
    total_assets_prior = balance.get('Total Assets', pd.Series([0]*num_years)).shift(1).iloc[latest]
    metrics['p/a'] = (net_income.iloc[latest] / total_assets_prior * 100) if total_assets_prior != 0 else 0
    
    # Price to Earnings (p/e)
    pe = info.get('trailingPE', float('inf'))
    metrics['p/e'] = pe
    
    # Earnings Yield (EY)
    metrics['EY'] = (net_income.iloc[latest] / (info.get('sharesOutstanding', 1) * info.get('regularMarketPrice', 0)) * 100) if info.get('regularMarketPrice', 0) != 0 else 0
    
    # Earnings Growth 5yr CAGR
    eps_values = income.get('Basic EPS', pd.Series([0]*num_years))[-min(5, num_years):]
    if pd.isnull(eps_values.iloc[0]):
        eps_values = income.get('Basic EPS', pd.Series([0]*num_years))[-min(4, num_years):]
    periods = len(eps_values) - 1
    metrics['Earnings Growth 5yr cagr'] = calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods) if periods > 0 else 0
    

    # Optional: Sales Growth 5yr CAGR
    revenue_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(5, num_years):]
    if pd.isnull(revenue_values.iloc[0]):
        revenue_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(4, num_years):]
    
    periods_revenue = len(revenue_values) - 1
    sales_cagr = calculate_cagr(revenue_values.iloc[0], revenue_values.iloc[-1], periods_revenue) if periods_revenue > 0 else 0
    metrics['Sales Growth 5yr cagr'] = sales_cagr
    # PEG
    metrics['PEG'] = (pe / metrics['Earnings Growth 5yr cagr']) if metrics['Earnings Growth 5yr cagr'] != 0 else float('inf')
    
    # No. shares (cr) - in crores
    shares_out = info.get('sharesOutstanding', 0)
    metrics['no. shares (cr)'] = shares_out / 1e7
    
    # Price to Sales (p/s)
    metrics['p/s'] = info.get('priceToSalesTrailing12Months', 0)
    
    # NFA + CWIP
    cwip = balance.get('Construction In Progress', pd.Series([0]*num_years)).iloc[latest]
    metrics['NFA + CWIP'] = net_fixed_assets.iloc[latest] + cwip
    
    # Capex = (NFA + CWIP end) - (NFA + CWIP start) + Dep
    if num_years >= 2:
        nfa_cwip_end = net_fixed_assets.iloc[latest] + cwip
        nfa_cwip_start = net_fixed_assets.iloc[latest-1] + balance.get('Construction In Progress', pd.Series([0]*num_years)).iloc[latest-1]
        metrics['Capex'] = nfa_cwip_end - nfa_cwip_start + dep.iloc[latest]
    else:
        metrics['Capex'] = abs(cashflow.get('Capital Expenditure', pd.Series([0]*num_years)).iloc[latest])  # Fallback
    
    # Free Cash Flow (FCF) = CFO - Capex
    metrics['FCF'] = metrics['CFO'] - metrics['Capex']
    
    # FCF%
    metrics['FCF%'] = (metrics['FCF'] / net_income.iloc[latest] * 100) if net_income.iloc[latest] != 0 else 0
    
    # Dividend Yield (DV)
    per_share_div = (dividends_paid.iloc[latest] / shares_out) if shares_out != 0 else 0
    metrics['DV'] = (per_share_div / info.get('regularMarketPrice', 0) * 100) if info.get('regularMarketPrice', 0) != 0 else 0
    
    # Mcap (cr)
    metrics['Mcap (cr)'] = info.get('marketCap', 0) / 1e7
    
    # d/e decreasing trend 5 yrs
    de_ratios = [balance.get('Total Debt', pd.Series([0]*num_years)).iloc[i] / balance.get('Stockholders Equity', pd.Series([1]*num_years)).iloc[i] for i in range(-min(5, num_years), 0)]
    
    metrics['d/e decreasing trend 5 yrs'] = all(de_ratios[j] > de_ratios[j+1] for j in range(len(de_ratios)-1)) if len(de_ratios) > 1 else False
    
    # Financial Analysis Criteria
    sales_values = income.get('Total Revenue', pd.Series([0]*num_years))[-min(6, num_years):]
    sales_periods = len(sales_values) - 1
    #sales_cagr = calculate_cagr(sales_values.iloc[0], sales_values.iloc[-1], sales_periods) if sales_periods > 0 else 0
    metrics['Sales cagr >15%'] = sales_cagr > 15
    metrics['npm >8%'] = metrics['NPM'] > 8
    metrics['Tax payout >25%'] = metrics['tax %'] > 25
    metrics['Interest coverage >3'] = metrics['Interest coverage'] > 3
    metrics['d/e <0.5'] = metrics['d/e'] < 0.5
    metrics['CFO >0'] = metrics['CFO'] > 0
    
    #metrics['net cash flow positive'] = cashflow.get('Net Change in Cash', pd.Series([0]*num_years)).iloc[latest] > 0
    metrics['cCFO > PAT'] = metrics['cCFO/cPAT'] > 1
    
    # Valuation Analysis
    metrics['p/e <10'] = pe < 10
    metrics['peg <1'] = metrics['PEG'] < 1
    metrics['EY >7%'] = metrics['EY'] > 7
    metrics['p/b <3'] = info.get('priceToBook', float('inf')) < 3
    metrics['DV >3%'] = metrics['DV'] > 3
    
    # Margin of Safety
    metrics['EY >7'] = metrics['EY'] > 7  # Duplicate
    metrics['sgr > Sales growth (very linear)'] = metrics['SSGR'] > sales_cagr
    metrics['FCF/CFO'] = (metrics['FCF'] / metrics['CFO']) if metrics['CFO'] != 0 else 0
    
    # Current Price
    metrics['Current Price'] = info.get('regularMarketPrice', 0)
    # Raw Financial Data
    # Raw Financial Data (corrected and enhanced columns with column existence check)
    metrics['Market Cap'] = info.get('marketCap', 0)  # In original currency
    metrics['Net Income'] = net_income.iloc[latest] if 'Net Income' in income.columns else 0
    metrics['Total Revenue'] = revenue.iloc[latest] if 'Total Revenue' in income.columns else 0
    metrics['Total Assets'] = balance['Total Assets'].iloc[latest] if 'Total Assets' in balance.columns else 0  # Fallback to sum if needed
    if metrics['Total Assets'] == 0 and 'Total Non Current Assets' in balance.columns and 'Current Assets' in balance.columns:
        metrics['Total Assets'] = (balance['Total Non Current Assets'].iloc[latest] + 
                                balance['Current Assets'].iloc[latest])
    metrics['Total Liabilities'] = balance['Total Liabilities Net Minority Interest'].iloc[latest] if 'Total Liabilities Net Minority Interest' in balance.columns else 0  # Use provided field
    if metrics['Total Liabilities'] == 0 and 'Total Non Current Liabilities Net Minority Interest' in balance.columns and 'Current Liabilities' in balance.columns:
        metrics['Total Liabilities'] = (balance['Total Non Current Liabilities Net Minority Interest'].iloc[latest] + 
                                    balance['Current Liabilities'].iloc[latest])
    metrics['Total Stockholders Equity'] = equity.iloc[latest] if 'Stockholders Equity' in balance.columns else 0
    
    metrics['Total Debt'] = (balance['Total Debt'].iloc[latest] if 'Total Debt' in balance.columns else balance['Long Term Debt'].iloc[latest] + balance['Current Debt'].iloc[latest] if 'Long Term Debt' in balance.columns and 'Current Debt' in balance.columns else 
                        balance['Total Non Current Liabilities Net Minority Interest'].iloc[latest] if 'Total Non Current Liabilities Net Minority Interest' in balance.columns else 0)
    metrics['Cash and Cash Equivalents'] = balance['Cash'].iloc[latest] if 'Cash' in balance.columns else balance['Other Current Assets'].iloc[latest] if 'Other Current Assets' in balance.columns else 0  # Fallback to Other Current Assets
    metrics['Current Assets'] = balance['Current Assets'].iloc[latest] if 'Current Assets' in balance.columns else 0
    if metrics['Current Assets'] == 0 and all(col in balance.columns for col in ['Other Current Assets', 'Hedging Assets Current', 'Assets Held For Sale Current', 'Prepaid Assets']):
        metrics['Current Assets'] = (balance['Other Current Assets'].iloc[latest] + 
                                    balance['Hedging Assets Current'].iloc[latest] + 
                                    balance['Assets Held For Sale Current'].iloc[latest] + 
                                    balance['Prepaid Assets'].iloc[latest])
    metrics['Current Liabilities'] = balance['Current Liabilities'].iloc[latest] if 'Current Liabilities' in balance.columns else 0
    metrics['Working Capital'] = balance['Working Capital'].iloc[latest] if 'Working Capital' in balance.columns else 0
    if metrics['Working Capital'] == 0 and all(col in balance.columns for col in ['Current Assets', 'Current Liabilities']):
        metrics['Working Capital'] = (metrics['Current Assets'] - metrics['Current Liabilities']) if metrics['Current Assets'] != 0 and metrics['Current Liabilities'] != 0 else 0
    metrics['Operating Cash Flow'] = metrics['CFO']  # Already calculated, stored raw
    metrics['Capital Expenditure'] = abs(cashflow['Capital Expenditure'].iloc[latest]) if 'Capital Expenditure' in cashflow.columns else 0
    metrics['Dividends Paid'] = dividends_paid.iloc[latest] if 'Cash Dividends Paid' in cashflow.columns else 0
    metrics['Depreciation & Amortization'] = dep.iloc[latest] if 'Depreciation And Amortization' in cashflow.columns else 0
    metrics['Interest Expense'] = interest_exp if 'Interest Expense' in income.columns else 0
    metrics['Income Tax Expense'] = income['Income Tax Expense'].iloc[latest] if 'Income Tax Expense' in income.columns else income['Tax Provision'].iloc[latest] if 'Tax Provision' in income.columns else 0
    metrics['Shares Outstanding'] = shares_out
    metrics['Current Price'] = info.get('regularMarketPrice', 0)  # Already calculated, stored raw
    metrics['Net Fixed Assets'] = net_fixed_assets.iloc[latest] if 'Net PPE' in balance.columns else 0
    metrics['Construction in Progress'] = cwip if 'Construction In Progress' in balance.columns else 0
    
    print(metrics['Total Debt'])
    print(metrics['Cash and Cash Equivalents'])
    metrics['Net Debt'] = metrics['Total Debt'] - metrics['Cash and Cash Equivalents'] if metrics['Total Debt'] != 0 and metrics['Cash and Cash Equivalents'] != 0 else 0
    
    # ROA and ROE Calculations (Average over 3-5 years)
    net_income_series = income.get('Net Income', pd.Series([0]*num_years))[-min(5, num_years):]
    total_assets_series = balance.get('Total Assets', pd.Series([0]*num_years))[-min(5, num_years):]
    equity_series = balance.get('Stockholders Equity', pd.Series([0]*num_years))[-min(5, num_years):]

    # Calculate yearly ROA and ROE
    years = min(5, num_years)
    roa_values = []
    roe_values = []
    for i in range(years):
        net_income = net_income_series.iloc[i] if not pd.isna(net_income_series.iloc[i]) else 0
        # Average Total Assets for ROA (current + previous) / 2
        if i > 0:
            avg_assets = (total_assets_series.iloc[i] + total_assets_series.iloc[i-1]) / 2
        else:
            avg_assets = total_assets_series.iloc[i]  # Fallback to current if no previous
        if avg_assets != 0:
            roa_values.append((net_income / avg_assets) * 100)
        
        # Average Equity for ROE (current + previous) / 2
        if i > 0:
            avg_equity = (equity_series.iloc[i] + equity_series.iloc[i-1]) / 2
        else:
            avg_equity = equity_series.iloc[i]  # Fallback to current if no previous
        if avg_equity != 0:
            roe_values.append((net_income / avg_equity) * 100)

    roa_values = [x for x in roa_values if pd.notnull(x)]
    roe_values = [x for x in roe_values if pd.notnull(x)]
    # Average ROA and ROE
    metrics['3-5yr Average ROA (%)'] = np.mean(roa_values) if roa_values else 0
    metrics['3-5yr Average ROE (%)'] = np.mean(roe_values) if roe_values else 0
    
    metrics['ROE'] = roe_values[latest]
    metrics['ROA'] = roa_values[latest]

    major_holders_dict = dict(stock.major_holders)['Value']
    price_targets = stock.analyst_price_targets
    metrics = {**metrics, **major_holders_dict, **price_targets}

    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])
    metrics_df.to_csv(f'yfin/{metrics['ticker']}.csv')
    return metrics



In [370]:
def complete_data_fmp(ticker):
    api_key = "XvrsN9KJGVVWKvmIpZ23BubrnrkozwYq"
    #print(f'getting data for {ticker} from financial modelling prep')
    #ticker = "AAPL"  # Example Indian stock
    data = get_peaceful_investing_data_fmp(ticker, api_key)
    print(f'got data for {ticker} from financial modelling prep')
    income, balance, cashflow, key_metrics, profile, quote = data
    metrics = compute_metrics_fmp(income, balance, cashflow, key_metrics, profile, quote)
    print(f'computed metrics for {ticker} from financial modelling prep')
    return metrics

def complete_yfinance_data(ticker):
    #ticker = "GHCL.NS"  # Indian stock
    stock = retrieve_data_from_yfinance(ticker)
    print(f'got data for {ticker} from yfinance')
    metrics=get_peaceful_investing_metrics_from_yfinance(stock)
    print(f'computed metrics for {ticker} from yfinance')
    return metrics


def get_any_ticker_data(ticker):
    try:
        metrics = complete_yfinance_data(ticker)
    except:
        try: 
            metrics =complete_data_fmp(ticker)
        except:pass
    return metrics

In [401]:
a2 = pd.read_excel('raw_tickers/indian_stonks.xlsx')
#a1 = pd.read_excel('raw_tickers/finviz_list_1_us_cheap_growing.xlsx')
list_of_stocks = list(a2['stock'])
do_everything_for_list_of_stocks(list_of_stocks, 'indian_stonks')   

got data for WSI.NS from yfinance
computed metrics for WSI.NS from yfinance
done ticker : WSI.NS
got data for VIPULLTD.NS from yfinance
computed metrics for VIPULLTD.NS from yfinance
done ticker : VIPULLTD.NS
got data for VINDHYATEL.NS from yfinance
computed metrics for VINDHYATEL.NS from yfinance
done ticker : VINDHYATEL.NS
got data for UTKARSHBNK.NS from yfinance
computed metrics for UTKARSHBNK.NS from yfinance
done ticker : UTKARSHBNK.NS
got data for UNITEDTEA.NS from yfinance
couldnt do : UNITEDTEA.NS
done ticker : UNITEDTEA.NS
got data for UNIONBANK.NS from yfinance
computed metrics for UNIONBANK.NS from yfinance
done ticker : UNIONBANK.NS
got data for UJJIVANSFB.NS from yfinance


/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_9760/562556452.py:11: RuntimeWarning: invalid value encountered in scalar power
  return ((end_value / start_value) ** (1 / periods) - 1) * 100


computed metrics for UJJIVANSFB.NS from yfinance
done ticker : UJJIVANSFB.NS
got data for TVTODAY.NS from yfinance
computed metrics for TVTODAY.NS from yfinance
done ticker : TVTODAY.NS
got data for TMB.NS from yfinance
computed metrics for TMB.NS from yfinance
done ticker : TMB.NS
got data for TIL.NS from yfinance


/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_9760/562556452.py:11: RuntimeWarning: invalid value encountered in scalar power
  return ((end_value / start_value) ** (1 / periods) - 1) * 100


computed metrics for TIL.NS from yfinance
done ticker : TIL.NS
got data for TFCILTD.NS from yfinance
computed metrics for TFCILTD.NS from yfinance
done ticker : TFCILTD.NS
got data for TATAMOTORS.NS from yfinance


...

computed metrics for CCCL.NS from yfinance
done ticker : CCCL.NS
got data for CAPITALSFB.NS from yfinance
computed metrics for CAPITALSFB.NS from yfinance
done ticker : CAPITALSFB.NS
got data for CANFINHOME.NS from yfinance
computed metrics for CANFINHOME.NS from yfinance
done ticker : CANFINHOME.NS
got data for CANBK.NS from yfinance
computed metrics for CANBK.NS from yfinance
done ticker : CANBK.NS
got data for BLBLIMITED.NS from yfinance
computed metrics for BLBLIMITED.NS from yfinance
done ticker : BLBLIMITED.NS
got data for BIRLAMONEY.NS from yfinance
computed metrics for BIRLAMONEY.NS from yfinance
done ticker : BIRLAMONEY.NS
got data for BFINVEST.NS from yfinance
computed metrics for BFINVEST.NS from yfinance
done ticker : BFINVEST.NS
got data for BCLIND.NS from yfinance
computed metrics for BCLIND.NS from yfinance
done ticker : BCLIND.NS
got data for BANKINDIA.NS from yfinance
computed metrics for BANKINDIA.NS from yfinance
done ticker : BANKINDIA.NS
got data for BANKBARODA.NS 

/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_9760/562556452.py:11: RuntimeWarning: invalid value encountered in scalar power
  return ((end_value / start_value) ** (1 / periods) - 1) * 100


computed metrics for ASHIMASYN.NS from yfinance
done ticker : ASHIMASYN.NS
got data for ARIHANTCAP.NS from yfinance
computed metrics for ARIHANTCAP.NS from yfinance
done ticker : ARIHANTCAP.NS
got data for ARENTERP.NS from yfinance
computed metrics for ARENTERP.NS from yfinance
done ticker : ARENTERP.NS
got data for ANMOL.NS from yfinance
computed metrics for ANMOL.NS from yfinance
done ticker : ANMOL.NS
got data for AMBIKCO.NS from yfinance
computed metrics for AMBIKCO.NS from yfinance
done ticker : AMBIKCO.NS
got data for ALMONDZ.NS from yfinance
computed metrics for ALMONDZ.NS from yfinance
done ticker : ALMONDZ.NS
got data for AIIL.NS from yfinance
computed metrics for AIIL.NS from yfinance
done ticker : AIIL.NS
got data for AHLEAST.NS from yfinance


/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_9760/562556452.py:11: RuntimeWarning: invalid value encountered in scalar power
  return ((end_value / start_value) ** (1 / periods) - 1) * 100


computed metrics for AHLEAST.NS from yfinance
done ticker : AHLEAST.NS
got data for AFIL.NS from yfinance
couldnt do : AFIL.NS
done ticker : AFIL.NS
got data for ABCAPITAL.NS from yfinance
computed metrics for ABCAPITAL.NS from yfinance
done ticker : ABCAPITAL.NS
got data for 20MICRONS.NS from yfinance
computed metrics for 20MICRONS.NS from yfinance
done ticker : 20MICRONS.NS
not done : UNITEDTEA.NS
not done : STEELCITY.NS
not done : PKTEA.NS
not done : NMDC.NS
not done : MOKSH.NS
not done : LAGNAM.NS
not done : JAIPURKURT.NS
not done : GLOBAL.NS
not done : GEEKAYWIRE.NS
not done : CSBBANK.NS
not done : ASPINWALL.NS
not done : AFIL.NS


In [400]:
a2

,stock,name,sector,industry
0,WSI.NS,W.S. Industries (India) Limited,Industrials,Electrical Equipment & Parts
1,VIPULLTD.NS,Vipul Limited,Real Estate,Real Estate—Diversified
2,VINDHYATEL.NS,Vindhya Telelinks Limited,Industrials,Engineering & Construction
3,UTKARSHBNK.NS,Utkarsh Small Finance Bank Limited,Financial Services,Banks—Regional
4,UNITEDTEA.NS,The United Nilgiri Tea Estates Company Limited,Consumer Defensive,Farm Products
...,...,...,...,...
130,AIIL.NS,Authum Investment & Infrastructure Limited,Financial Services,Capital Markets
131,AHLEAST.NS,Asian Hotels (East) Limited,Consumer Cyclical,Lodging
132,AFIL.NS,AKME FINTRADE (INDIA) LTD,Financial Services,Credit Services
133,ABCAPITAL.NS,Aditya Birla Capital Limited,Financial Services,Financial Conglomerates


In [ ]:
a1 = pd.read_excel('raw_tickers/finviz_list_1_us_cheap_growing.xlsx')
list_of_stocks = list(a1['Ticker'])
do_everything_for_list_of_stocks(list_of_stocks, 'finviz_list_1_cheap_us_growing')

In [391]:
def do_everything_for_list_of_stocks(list_of_stocks, final_file):

    for st in list_of_stocks:
        try:
            metrics = complete_yfinance_data(st)
        except:
            print(f'couldnt do : {st}')
        time.sleep(1)
        print(f'done ticker : {st}')

    list_dfs = []
    for st in list_of_stocks:
        try:
            p1 =pd.read_csv(f'yfin/{st}.csv')
            columns_order = p1['Unnamed: 0'].values
            p1= p1.fillna(-1)
            piv_df = p1.pivot_table(index=None, columns='Unnamed: 0', values='Value', aggfunc='first').reset_index(drop=True)[columns_order]
            list_dfs.append(piv_df)
        except:print(f"not done : {st}")
    new_df = pd.concat(list_dfs)
    #finviz_list_1_cheap_us_growing
    new_df.to_csv(f'combined_stocks/{final_file}.csv')


In [361]:
net_income = income.get('Net Income', pd.Series([0]*num_years))
dividends_paid = cashflow.get('Cash Dividends Paid', pd.Series([0]*num_years)).abs()
dpr_3yr = [(dividends_paid.iloc[i] / net_income.iloc[i] * 100) for i in range(-min(3, num_years), 0) if net_income.iloc[i] != 0]
#metrics['DPR'] = np.mean(dpr_3yr) if dpr_3yr else 0

In [372]:
dpr_3yr

[np.float64(0.0), np.float64(0.0), np.float64(0.0)]

In [357]:
dep_3yr

[np.float64(0.10016147477438504),
 np.float64(0.12766838400620548),
 np.float64(0.11877481453750137)]

In [ ]:
#!mkdir combined_stocks


In [417]:
stock = retrieve_data_from_yfinance('MUTHOOTMF.NS')

In [421]:
metrics=get_peaceful_investing_metrics_from_yfinance(stock)

nfat_3yr : [np.float64(nan), np.float64(14.435387015934559), np.float64(14.068404766797984)]
81013440000.0
443240000.0


/var/folders/jg/dt8f9z8s7f5gtvdq0cqqmc2h0000gn/T/ipykernel_9760/4100009545.py:11: RuntimeWarning: invalid value encountered in scalar power
  return ((end_value / start_value) ** (1 / periods) - 1) * 100


In [419]:
metrics

{'ticker': 'MUTHOOTMF.NS',
 'ROCE': np.float64(-4.388297008899411),
 'ROCE (3yr avg)': np.float64(3.3926812260609887),
 'NFAT': np.float64(14.068404766797984),
 'NFAT (3yr avg)': np.float64(nan),
 'NPM': np.float64(8.731074415392435),
 'DPR': np.float64(0.0),
 'Retention Ratio': np.float64(100.0),
 'Dep': np.float64(19.949802600036307),
 'SSGR': np.float64(nan),
 'Av NPM (over 3 years)': np.float64(8.731074415392435),
 'Au NFA/T (over 3 years)': np.float64(nan),
 'Av Dep%NFA (over 3 years)': np.float64(19.949802600036307),
 'Au Retention ratio (over 3 years)': np.float64(100.0),
 'd/e': np.float64(2.88884704915788),
 'Interest coverage': np.float64(6.871654009802337),
 'tax %': np.float64(22.97683659623958),
 'cPAT': np.float64(4383470000.0),
 'CFO': np.float64(13820280000.0),
 'cCFO': np.float64(-33061210000.0),
 'cCFO/cPAT': np.float64(-7.542246211334856),
 'p/a': np.float64(-1.9199194217610596),
 'p/e': inf,
 'EY': np.float64(-8.72294006872397),
 'Earnings Growth 5yr cagr': np.float

In [375]:
cashflow.get('Depreciation And Amortization', pd.Series([0]*num_years))

4           NaN
3    18840000.0
2    23385000.0
1    24359000.0
0    24112000.0
Name: Depreciation And Amortization, dtype: float64

In [376]:
revenue = income.get('Total Revenue', pd.Series([0]*num_years))
nfat_values = []
for i in range(1, num_years):
    avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.iloc[i-1]) / 2
    nfat_year = revenue.iloc[i] / avg_nfa_year if avg_nfa_year != 0 else 0
    nfat_values.append(nfat_year)
if num_years == 1:
    avg_nfa_year = net_fixed_assets.iloc[latest]
    nfat_year = revenue.iloc[latest] / avg_nfa_year if avg_nfa_year != 0 else 0
    nfat_values.append(nfat_year)
metrics['NFAT'] = nfat_values[-1] if nfat_values else 0  # Latest year's NFAT
nfat_3yr = nfat_values[-min(3, len(nfat_values)):]
    

In [381]:
net_fixed_assets = balance.get('Net PPE', pd.Series([0]*num_years))
dep = cashflow.get('Depreciation And Amortization', pd.Series([0]*num_years))
dep_3yr = []
for i in range(-min(3, num_years), 0):
    avg_nfa_year = (net_fixed_assets.iloc[i] + net_fixed_assets.shift(1).iloc[i]) / 2 if num_years > 1 and i > -num_years + 1 else net_fixed_assets.iloc[i]
    dep_year = min((abs(dep.iloc[i]) / avg_nfa_year * 100) if avg_nfa_year != 0 else 0, 100)  # Use abs for dep, cap at 100%
    #print(avg_nfa_year)
    if dep_year > 100:
        print(f"Warning: Dep % for year {i} exceeded 100%; capped to 100%. Check data for {ticker}.")
    dep_3yr.append(dep_year)
    

In [389]:
dep_3yr

[100, 100, 100]

In [383]:
net_fixed_assets,dep

(4          NaN
 3     820000.0
 2     573000.0
 1     371000.0
 0    1257000.0
 Name: Net PPE, dtype: float64,
 4           NaN
 3    18840000.0
 2    23385000.0
 1    24359000.0
 0    24112000.0
 Name: Depreciation And Amortization, dtype: float64)

In [291]:
find = 'eps'
print([x for x in balance.columns if find in x.lower()])
print([x for x in income.columns if find in x.lower()])
print([x for x in cashflow.columns if find in x.lower()])

[]
['Diluted EPS', 'Basic EPS']
[]


In [ ]:
# Earnings Growth 5yr CAGR
num_years = len(income)
eps_values = income.get('Basic EPS', pd.Series([0]*num_years))[-min(6, num_years):]
periods = len(eps_values) - 1
metrics['Earnings Growth 5yr cagr'] = calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods) if periods > 0 else 0

In [298]:
calculate_cagr(eps_values.iloc[0], eps_values.iloc[-1], periods)


np.float64(nan)

In [300]:
eps_values.iloc[-1]

np.float64(2.56)

In [403]:
income = stock.financials.transpose()  # Transpose to rows=years, columns=items; recent first
balance = stock.balance_sheet.transpose()
cashflow = stock.cashflow.transpose()
info = stock.info  # Dict for quote/profile/metrics

# Earnings history for EPS/Revenue (annual)
earnings = stock.earnings  # pd.DataFrame with Revenue, Earnings

# Reset index to make date a column; sort oldest first for calculations
income = income.reset_index().rename(columns={'index': 'Date'}).sort_values('Date')
balance = balance.reset_index().rename(columns={'index': 'Date'}).sort_values('Date')
cashflow = cashflow.reset_index().rename(columns={'index': 'Date'}).sort_values('Date')

/Users/dhruvsabharwal/Documents/personal/financial_analysis/venv/lib/python3.13/site-packages/yfinance/scrapers/fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


In [404]:
balance

,Date,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Preferred Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Other Equity Interest,Gains Losses Not Affecting Retained Earnings,Other Equity Adjustments,Treasury Stock,Retained Earnings,Additional Paid In Capital,Capital Stock,Common Stock,Preferred Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Tradeand Other Payables Non Current,Non Current Deferred Liabilities,Non Current Deferred Taxes Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Long Term Debt,Current Liabilities,Other Current Liabilities,Current Deferred Liabilities,Current Deferred Revenue,Current Debt And Capital Lease Obligation,Current Capital Lease Obligation,Current Debt,Other Current Borrowings,Payables And Accrued Expenses,Current Accrued Expenses,Payables,Other Payable,Total Tax Payable,Income Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Non Current Deferred Assets,Non Current Deferred Taxes Assets,Goodwill And Other Intangible Assets,Other Intangible Assets,Goodwill,Net PPE,Accumulated Depreciation,Gross PPE,Other Properties,Machinery Furniture Equipment,Current Assets,Other Current Assets,Restricted Cash,Prepaid Assets,Inventory,Finished Goods,Receivables,Other Receivables,Duefrom Related Parties Current,Taxes Receivable,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Financial
4,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26024000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26024000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,711000.0,NaN,NaN,NaN,NaN,NaN,NaN,392000.0,392000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61542000.0
3,2021-12-31,NaN,39753546.0,39753546.0,5270000.0,98308000.0,98890000.0,115375000.0,125832000.0,4688000.0,98308000.0,27524000.0,126069000.0,125832000.0,125832000.0,-79000.0,-165000.0,-165000.0,0.0,60559000.0,37439000.0,28078000.0,554000.0,27524000.0,60945000.0,3901000.0,1033000.0,286000.0,286000.0,2582000.0,2345000.0,237000.0,57044000.0,11084000.0,3690000.0,3690000.0,2688000.0,2343000.0,345000.0,345000.0,39582000.0,900000.0,38682000.0,5394000.0,8148000.0,8148000.0,25140000.0,1.867770e+08,14358000.0,3211000.0,72000.0,72000.0,NaN,NaN,NaN,11075000.0,-1232000.0,12307000.0,10739000.0,1568000.0,172419000.0,298000.0,664000.0,3646000.0,81441000.0,81441000.0,23172000.0,1407000.0,3165000.0,564000.0,18036000.0,-151000.0,18187000.0,63198000.0,NaN,63198000.0,63198000.0
2,2022-12-31,NaN,40684546.0,40684546.0,148207000.0,195165000.0,195372000.0,155484000.0,195165000.0,148000000.0,195165000.0,NaN,195165000.0,195165000.0,195165000.0,-81000.0,804000.0,804000.0,231000.0,83590000.0,109049000.0,2034000.0,2034000.0,0.0,223435000.0,120797000.0,2894000.0,472000.0,472000.0,117431000.0,117431000.0,NaN,102638000.0,21721000.0,2001000.0,2001000.0,30776000.0,30569000.0,207000.0,207000.0,48140000.0,1500000.0,46640000.0,10925000.0,4142000.0,4142000.0,31573000.0,4.186000e+08,160478000.0,3182000.0,75000.0,75000.0,0.0,NaN,0.0,157221000.0,-2595000.0,159816000.0,157798000.0,2018000.0,258122000.0,1050000.0,1545000.0,1434000.0,78338000.0,78338000.0,32224000.0,1067000.0,3814000.0,201000.0,27142000.0,-237000.0,27379000.0,143531000.0,NaN,143531000.0,143531000.0
1,2023-12-31,294029.0,40781880.0,41075909.0,403237000.0,269463000.0,290416000.0,187033000.0,269463000.0,403237000.0,290416000.0,NaN,290416000.0,290416000.0,290416000.0,NaN,526000.0,526000.0,1594000.0,177698000.0,111736000.0,2050000.0,2050000.0,NaN,556493000.0,350719000.0,3302000.0,3795000.0,3795000.0,343622

In [186]:
stock.analyst_price_targets

{'current': 585.3, 'high': 900.0, 'low': 651.0, 'mean': 775.5, 'median': 775.5}

In [201]:
dict(stock.major_holders)['Value']

insidersPercentHeld              0.35814
institutionsPercentHeld          0.23611
institutionsFloatPercentHeld     0.36785
institutionsCount               44.00000
Name: Value, dtype: float64

In [205]:
stock.financials

,2025-03-31,2024-03-31,2023-03-31,2022-03-31,2021-03-31
Tax Effect Of Unusual Items,1.096607e+08,4.853763e+08,2.068249e+08,-8.598260e+06,NaN
Tax Rate For Calcs,2.514000e-01,1.933000e-01,2.373210e-01,2.662000e-01,NaN
Normalized EBITDA,9.197700e+09,8.663700e+09,1.544840e+10,7.760000e+09,NaN
Total Unusual Items,4.362000e+08,2.511000e+09,8.715000e+08,-3.230000e+07,NaN
Total Unusual Items Excluding Goodwill,4.362000e+08,2.511000e+09,8.715000e+08,-3.230000e+07,NaN
Net Income From Continuing Operation Net Minority Interest,6.241500e+09,7.939000e+09,1.115800e+10,4.459500e+09,NaN
Reconciled Depreciation,1.115400e+09,1.021000e+09,1.345600e+09,1.318700e+09,NaN
Reconciled Cost Of Revenue,1.757940e+10,2.074580e+10,2.526800e+10,1.820820e+10,NaN
EBITDA,9.633900e+09,1.117470e+10,1.631990e+10,7.727700e+09,NaN
EBIT,8.518500e+09,1.015370e+10,1.497430e+10,6.409000e+09,NaN


In [ ]:
cashflow
#pd.options.display.max_columns = 100

,Date,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Issuance Of Capital Stock,Capital Expenditure,End Cash Position,Other Cash Adjustment Outside Changein Cash,Beginning Cash Position,Changes In Cash,Financing Cash Flow,Net Other Financing Charges,Interest Paid Cff,Cash Dividends Paid,Net Common Stock Issuance,Common Stock Payments,Common Stock Issuance,Net Issuance Payments Of Debt,Net Short Term Debt Issuance,Short Term Debt Payments,Short Term Debt Issuance,Net Long Term Debt Issuance,Long Term Debt Payments,Long Term Debt Issuance,Investing Cash Flow,Interest Received Cfi,Dividends Received Cfi,Net Investment Purchase And Sale,Sale Of Investment,Purchase Of Investment,Net Business Purchase And Sale,Sale Of Business,Net PPE Purchase And Sale,Sale Of PPE,Purchase Of PPE,Operating Cash Flow,Taxes Refund Paid,Change In Working Capital,Change In Other Current Liabilities,Change In Other Current Assets,Change In Payable,Change In Inventory,Change In Receivables,Other Non Cash Items,Stock Based Compensation,Provisionand Write Offof Assets,Depreciation And Amortization,Amortization Cash Flow,Depreciation,Gain Loss On Investment Securities,Net Foreign Currency Exchange Gain Loss,Gain Loss On Sale Of PPE,Gain Loss On Sale Of Business,Net Income From Continuing Operations
4,2021-03-31,NaN,0.0,NaN,2.460000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-49000000.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,-2.275300e+09,NaN,NaN,NaN,2.460000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-31,3.059700e+09,NaN,-2.494100e+09,1.000000e+09,50600000.0,-3.374500e+09,2.437500e+09,NaN,3.497000e+08,2.087800e+09,-9.807000e+08,900000.0,-652900000.0,-5.200000e+08,50600000.0,NaN,50600000.0,1.730000e+08,1.667100e+09,NaN,1.667100e+09,-1.494100e+09,-2.494100e+09,1.000000e+09,-3.365700e+09,4000000.0,500000.0,4.300000e+06,4.300000e+06,0.000000e+00,0.000000e+00,0.000000e+00,-3.374500e+09,0.0,-3.374500e+09,6.434200e+09,-2.049200e+09,-2.407600e+09,3.259100e+09,-1.117700e+09,268100000.0,-3.813400e+09,-1.030600e+09,7.102000e+08,400000.0,0.0,1.318700e+09,NaN,1.318700e+09,-5600000.0,35200000.0,28400000.0,0.0,8.811700e+09
2,2023-03-31,4.986000e+09,NaN,-3.424000e+09,0.000000e+00,35300000.0,-3.575600e+09,1.612100e+09,200000.0,2.437500e+09,-8.256000e+08,-5.299600e+09,-100000.0,-446900000.0,-1.426800e+09,35300000.0,NaN,35300000.0,-3.424000e+09,-2.067200e+09,-2.067200e+09,NaN,-1.356800e+09,-1.356800e+09,0.000000e+00,-4.087600e+09,19700000.0,1300000.0,-3.541500e+09,3.352020e+10,-3.706170e+10,2.932700e+09,2.932700e+09,-3.499800e+09,75800000.0,-3.575600e+09,8.561600e+09,-4.432400e+09,-3.110400e+09,-7.300000e+07,1.306000e+08,-680200000.0,-1.910400e+09,-6.563000e+08,4.454000e+08,132900000.0,-1900000.0,1.345600e+09,NaN,1.345600e+09,-168900000.0,22500000.0,162900000.0,-797800000.0,1.496370e+10
1,2024-03-31,6.905200e+09,NaN,-1.511600e+09,NaN,51500000.0,-1.065100e+09,4.873000e+08,100000.0,1.234000e+09,-7.468000e+08,-3.376700e+09,NaN,-235700000.0,-1.664600e+09,51500000.0,NaN,51500000.0,-1.511600e+09,0.000000e+00,0.000000e+00,NaN,-1.511600e+09,-1.511600e+09,NaN,-5.340400e+09,107300000.0,1800000.0,-4.384800e+09,2.447000e+10,-2.885480e+10,0.000000e+00,0.000000e+00,-1.064700e+09,400000.0,-1.065100e+09,7.970300e+09,-1.924400e+09,1.634900e+09,2.175000e+08,4.060000e+07,-227900000.0,1.212600e+09,3.870000e+08,-2.350000e+09,12100000.0,0.0,1.021000e+09,18700000.0,1.021000e+09,-322000000.0,-36600000.0,19600000.0,0.0,9.915700e+09
0,2025-03-31,3.229900e+09,NaN,-9.943000e+08,NaN,11000000.0,-3.148900e+09,9.863000e+08,NaN,4.873000e+08,4.990000e+08,-2.303200e+09,NaN,-160600000.0,-1.143500e+09,11000000.0,NaN,11000000.0,-9.943000e+08,0.000000e+00,0.000000e+00,NaN,-9.943000e+08,-9.943000e+08,NaN,-3.576600e+09,317300000.0,2000000.0,-8.770000e+08,2.210210e+10,-2.297910e+10,NaN,NaN,-3.018900e+09,130000000.0,-3.148900e+09,6.378800e+09,-2.254600e+09,-1.753000e+08,1.350000e+07,9.830000e+07,-278800000.0,2.542000e+08,-

In [144]:
income

,Date,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,Interest Expense,Interest Income,Normalized Income,Net Income From Continuing And Discontinued Operation,Total Expenses,Rent Expense Supplemental,Diluted Average Shares,Basic Average Shares,Diluted EPS,Basic EPS,Diluted NI Availto Com Stockholders,Net Income Common Stockholders,Otherunder Preferred Stock Dividend,Net Income,Minority Interests,Net Income Including Noncontrolling Interests,Net Income Discontinuous Operations,Net Income Continuous Operations,Tax Provision,Pretax Income,Other Non Operating Income Expenses,Special Income Charges,Other Special Charges,Write Off,Net Non Operating Interest Income Expense,Total Other Finance Cost,Interest Expense Non Operating,Interest Income Non Operating,Operating Income,Operating Expense,Other Operating Expenses,Depreciation And Amortization In Income Statement,Amortization,Depreciation Income Statement,Selling General And Administration,Selling And Marketing Expense,General And Administrative Expense,Rent And Landing Fees,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
4,2021-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.928000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-31,-8.598260e+06,0.266200,7.760000e+09,-3.230000e+07,-3.230000e+07,4.459500e+09,1.318700e+09,1.820820e+10,7.727700e+09,6.409000e+09,-472900000.0,429000000.0,3900000.0,4.483202e+09,6.500500e+09,2.402020e+10,28700000.0,95545654.0,95253697.0,68.02,68.24,6.500500e+09,6.500500e+09,0.0,6.500500e+09,0.0,6.500500e+09,2.041000e+09,4.459500e+09,1.520500e+09,5.980000e+09,30400000.0,-3.740000e+07,3.410000e+07,3300000.0,-472900000.0,47800000.0,429000000.0,3900000.0,6.501200e+09,5.812000e+09,1.036000e+09,8.692000e+08,27700000.0,8.415000e+08,2.818800e+09,2.506600e+09,312200000.0,28700000.0,1.231320e+10,1.820820e+10,3.052140e+10,3.052140e+10
2,2023-03-31,2.068249e+08,0.237321,1.544840e+10,8.715000e+08,8.715000e+08,1.115800e+10,1.345600e+09,2.526800e+10,1.631990e+10,1.497430e+10,-367800000.0,344300000.0,18900000.0,1.049332e+10,1.141570e+10,3.130930e+10,26500000.0,95081527.0,95081527.0,120.06,120.06,1.141570e+10,1.141570e+10,0.0,1.141570e+10,0.0,1.141570e+10,2.577000e+08,1.115800e+10,3.472000e+09,1.463000e+10,31100000.0,7.040000e+08,-7.041000e+08,100000.0,-367800000.0,42400000.0,344300000.0,18900000.0,1.408540e+10,6.041300e+09,1.175300e+09,9.422000e+08,13800000.0,9.284000e+08,2.626500e+09,2.328500e+09,298000000.0,26500000.0,2.012670e+10,2.526800e+10,4.539470e+10,4.539470e+10
1,2024-03-31,4.853763e+08,0.193300,8.663700e+09,2.511000e+09,2.511000e+09,7.939000e+09,1.021000e+09,2.074580e+10,1.117470e+10,1.015370e+10,-115800000.0,239100000.0,150500000.0,5.913376e+09,7.939000e+09,2.687700e+10,35800000.0,95273279.0,95155762.0,83.33,83.43,7.939000e+09,7.939000e+09,0.0,7.939000e+09,0.0,7.939000e+09,0.000000e+00,7.939000e+09,1.975600e+09,9.914600e+09,26400000.0,2.190800e+09,-2.190800e+09,0.0,-115800000.0,27200000.0,239100000.0,150500000.0,7.484800e+09,6.131200e+09,1.277400e+09,1.021000e+09,18700000.0,1.002300e+09,2.748100e+09,2.406600e+09,341500000.0,35800000.0,1.361600e+10,2.074580e+10,3.436180e+10,3.436180e+10
0,2025-03-31,1.096607e+08,0.251400,9.197700e+09,4.362000e+08,4.362000e+08,6.241500e+09,1.115400e+09,1.757940e+10,9.633900e+09,8.518500e+09,160200000.0,157800000.0,335000000.0,5.914961e+09,6.241500e+09,2.408840e+10,50300000.0,95523668.0,95285560.0,65.34,65.50,6.241500e+09,6.241500e+09,0.0,6.241500e+09,0.0,6.241500e+09,NaN,6.241500e+09,2.119200e+09,8.360700e+09,17600000.0,6.000000e+06,-6.000000e+06,0.0,160200000.0,17000000.0,157800000.0,335000000.0,7.675600e+09,6.509000e+09,1.321100e+09,1.115400e+09,24900000.0,1.090500e+

In [173]:
balance

,Date,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Other Equity Interest,Treasury Stock,Retained Earnings,Additional Paid In Capital,Capital Stock,Common Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Other Non Current Liabilities,Non Current Deferred Revenue,Non Current Deferred Taxes Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Long Term Debt,Long Term Provisions,Current Liabilities,Other Current Liabilities,Current Debt And Capital Lease Obligation,Current Capital Lease Obligation,Current Debt,Pensionand Other Post Retirement Benefit Plans Current,Current Provisions,Payables,Other Payable,Dividends Payable,Total Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Non Current Prepaid Assets,Investmentin Financial Assets,Held To Maturity Securities,Available For Sale Securities,Investment Properties,Goodwill And Other Intangible Assets,Other Intangible Assets,Net PPE,Accumulated Depreciation,Gross PPE,Construction In Progress,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Properties,Current Assets,Other Current Assets,Hedging Assets Current,Assets Held For Sale Current,Restricted Cash,Prepaid Assets,Inventory,Finished Goods,Work In Process,Raw Materials,Other Receivables,Taxes Receivable,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial
4,2021-03-31,NaN,NaN,7.331300e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.090000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-31,95350786.0,95350786.0,5.238700e+09,7.875600e+09,2.944700e+10,3.714760e+10,9.812000e+09,2.944700e+10,201100000.0,2.947310e+10,3.371910e+10,2.947310e+10,2.947310e+10,54800000.0,0.0,2.793130e+10,270500000.0,953500000.0,953500000.0,2.048060e+10,8.579500e+09,-100000.0,0.0,4.078400e+09,4.425000e+09,179000000.0,4.246000e+09,76200000.0,1.190110e+10,14000000.0,3.450600e+09,22100000.0,3.428500e+09,0.0,119100000.0,4.059300e+09,440300000.0,45200000.0,145400000.0,3.428400e+09,4.995370e+10,2.824060e+10,200000.0,153800000.0,165500000.0,400000.0,165100000.0,0.0,26100000.0,26100000.0,2.769390e+10,-5.570000e+09,3.326390e+10,2.977900e+09,2.286400e+09,1.888530e+10,2.212100e+09,6.743600e+09,158600000.0,2.171310e+10,52900000.0,0.0,5.950400e+09,116500000.0,1.282900e+09,9.442300e+09,1.534200e+09,279200000.0,7.628900e+09,131900000.0,0.0,2.294400e+09,-5700000.0,2.300100e+09,2.441800e+09,6.000000e+06,2.435800e+09,2.133700e+09,302100000.0
2,2023-03-31,95585786.0,95585786.0,2.241200e+09,3.613600e+09,3.953950e+10,4.303310e+10,2.484040e+10,3.953950e+10,138400000.0,3.955790e+10,4.198410e+10,3.955790e+10,3.955790e+10,145200000.0,53500000.0,3.791350e+10,346300000.0,955900000.0,955900000.0,1.177620e+10,5.043600e+09,-200000.0,NaN,2.435000e+09,2.551100e+09,124900000.0,2.426200e+09,57700000.0,6.732600e+09,347100000.0,1.062500e+09,13500000.0,1.049000e+09,0.0,120600000.0,2.694200e+09,347100000.0,51400000.0,0.0,2.295700e+09,5.133410e+10,1.976110e+10,-200000.0,119000000.0,149700000.0,400000.0,149300000.0,NaN,18400000.0,18400000.0,1.900070e+10,-5.537700e+09,2.453840e+10,1.704600e+09,1.854000e+08,1.624490e+10,1.662400e+09,4.582500e+09,158600000.0,3.157300e+10,62900000.0,0.0,1.617860e+10,163700000.0,2.456000e+08,7.723300e+09,1.308500e+09,72100000.0,6.342700e+09,13500000.0,0.0,2.184100e+09,0.0,2.184100e+09,5.001300